### Matrix consturcted via Qiskit gate composition

In [7]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, library
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
import matplotlib.pyplot as plt

parameters = [Parameter(f'θ{i}') for i in range(6)]

rx_0 = library.RXGate(parameters[0])
ry_0 = library.RYGate(parameters[1])
rz_0 = library.RZGate(parameters[2])

# Create a function to get the combined matrix for given parameter values
def get_u_matrix(theta_values):
    """
    Compute U = RZ(θ2) @ RY(θ1) @ RX(θ0)
    """
    rx_matrix = library.RXGate(theta_values[0]).to_matrix()
    ry_matrix = library.RYGate(theta_values[1]).to_matrix()
    rz_matrix = library.RZGate(theta_values[2]).to_matrix()
    return rz_matrix @ ry_matrix @ rx_matrix

# Example: evaluate at specific parameter values
example_params = [np.pi/4, np.pi/3, np.pi/6]
u_0 = get_u_matrix(example_params)

# Print out the transformation matrix
print("U matrix at θ = [π/4, π/3, π/6]:")
print(u_0)
print(f"\nMatrix shape: {u_0.shape}")

# Alternative: Use a quantum circuit to build and evaluate the operator
def get_u_circuit(theta_values):
    qc = QuantumCircuit(1)
    qc.rx(theta_values[0], 0)
    qc.ry(theta_values[1], 0)
    qc.rz(theta_values[2], 0)
    return Operator(qc).data

# Verify both methods give the same result
u_circuit = get_u_circuit(example_params)
print("Verification - matrices are equal:", np.allclose(u_0, u_circuit))

U matrix at θ = [π/4, π/3, π/6]:
[[ 0.82236317-0.02226003j -0.5319757 -0.20056212j]
 [ 0.5319757 -0.20056212j  0.82236317+0.02226003j]]

Matrix shape: (2, 2)
Verification - matrices are equal: True


### Manually computed matrix (verified with numpy, produces same result)

In [9]:
def get_u(theta_0, theta_1, theta_2):
    # 2x2 matrix [a, b; c, d]
    # a = exp(-i*theta_2/2) * cos(theta_1/2) * cos(theta_0/2) + exp(-i*theta_2/2) * sin(theta_1/2) * sin(theta_0/2) * i
    a = (np.cos(theta_1 / 2) * np.cos(theta_0 / 2) * np.exp(-1j * theta_2 / 2) +
         1j * np.sin(theta_1 / 2) * np.sin(theta_0 / 2) * np.exp(-1j * theta_2 / 2))
    # b = -exp(-i*theta_2/2) * sin(theta_1/2) * cos(theta_0/2) - exp(-i*theta_2/2) * cos(theta_1/2) * sin(theta_0/2) * i
    b = (-np.sin(theta_1 / 2) * np.cos(theta_0 / 2) * np.exp(-1j * theta_2 / 2) -
         1j * np.cos(theta_1 / 2) * np.sin(theta_0 / 2) * np.exp(-1j * theta_2 / 2))
    # c = exp(i*theta_2/2) * sin(theta_1/2) * cos(theta_0/2) - exp(i*theta_2/2) * cos(theta_1/2) * sin(theta_0/2) * i
    c = (np.sin(theta_1 / 2) * np.cos(theta_0 / 2) * np.exp(1j * theta_2 / 2) -
         1j * np.cos(theta_1 / 2) * np.sin(theta_0 / 2) * np.exp(1j * theta_2 / 2))
    # d = exp(i*theta_2/2) * cos(theta_1/2) * cos(theta_0/2) - exp(i*theta_2/2) * sin(theta_1/2) * sin(theta_0/2) * i
    d = (np.cos(theta_1 / 2) * np.cos(theta_0 / 2) * np.exp(1j * theta_2 / 2) -
         1j * np.sin(theta_1 / 2) * np.sin(theta_0 / 2) * np.exp(1j * theta_2 / 2))
    return np.array([[a, b], [c, d]])

# Example usage
theta_0 = np.pi / 4
theta_1 = np.pi / 3
theta_2 = np.pi / 6
u_matrix = get_u(theta_0, theta_1, theta_2)
print("U matrix from get_u function:")
print(u_matrix)

U matrix from get_u function:
[[ 0.82236317-0.02226003j -0.5319757 -0.20056212j]
 [ 0.5319757 -0.20056212j  0.82236317+0.02226003j]]


In [1]:
# TODO, not working yet

# inputs = ['00', '01', '10', '11']

# and_outputs = [0,0,0,1]  # Corresponding outputs for AND gate

# def apply_u(theta_0, theta_1, theta_2):
#     u_matrix = get_u(theta_0, theta_1, theta_2)
#     # Initial state |0>
#     initial_state = np.array([1, 0])
#     # Apply U to the initial state
#     final_state = u_matrix @ initial_state
#     return final_state

# def get_prob(expected_output):
#     def calc_prob(theta_0, theta_1, theta_2):
#         # Apply U to the initial state
#         final_state = apply_u(theta_0, theta_1, theta_2)
#         # Probability of measuring |expected_output>
#         prob_1 = np.abs(final_state[expected_output])**2
#         return prob_1
#     return calc_prob

# for inp, out in zip(inputs, and_outputs):
#     prob_func = get_prob(out)
#     # Optimize the parameters to maximize the probability
#     res = minimize(lambda x: -prob_func(x[0], x[1], x[2]), [0, 0, 0], bounds=[(0, 2*np.pi)]*3)
#     optimized_params = res.x
#     final_state = apply_u(optimized_params[0], optimized_params[1], optimized_params[2])
#     max_prob = np.abs(final_state[out])**2
    
#     print(f"Input: {inp}, Expected Output: {out}, Optimized Parameters: {res.x}, Max Probability: {-res.fun}, Success: {res.success}")
#     print(f"Final State: {final_state}\n")